### 7장 6번
AdaBoost가 과소적합되었을 때 학습률(learning_rate)을 높이거나 예측기의 개수(n_estimators)를 늘려야 함.
그 이유는 learning_rate가 너무 낮으면 가중치 업데이트가 작아져서 모델이 충분히 학습하지 못해 과소적합이 발생할 수 있음.
또한, 예측기 개수를 늘리면 학습을 더 많이 진행해서 과소적합 문제를 해결할 수 있음.

### 7장 7번

그래이디언트 부스팅이 과대적합되었을 때 학습률을 낮추거나 예측기의 개수를 줄여야 함.

### 7장 8번
아래 코드 설명 : mnist 데이터를 불러온 후 개별 모델을 훈련한 것의 정확도와 앙상블 기법을 사용해서 훈련한 것의 정확도를 비교해보았습니다.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# MNIST 데이터 불러오기
mnist = fetch_openml('mnist_784', as_frame=False)
X, y = mnist.data, mnist.target

# 데이터를 숫자로 변환 (y가 문자열이므로 변환 필요)
y = y.astype(int)

# 전체 데이터 개수: 70,000개
# 1. 훈련 (50,000개) + 검증 (10,000개) + 테스트 (10,000개) 세트로 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2857, random_state=42, stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 데이터 크기 확인
print(f"훈련 세트 크기: {X_train.shape}, 검증 세트 크기: {X_valid.shape}, 테스트 세트 크기: {X_test.shape}")

훈련 세트 크기: (50001, 784), 검증 세트 크기: (9999, 784), 테스트 세트 크기: (10000, 784)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_train)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

ext_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
ext_clf.fit(X_train, y_train)

y_pred_ext = ext_clf.predict(X_train)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

svm_clf = make_pipeline(StandardScaler(), LinearSVC(gamma='auto'))
svm_clf.fit(X_train, y_train)

y_pred_svm = svm_clf.predict(X_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml

# 1. 데이터 불러오기 및 분할
mnist = fetch_openml('mnist_784', as_frame=False)
X, y = mnist.data, mnist.target.astype(int)

# 훈련 (50,000개), 검증 (10,000개), 테스트 (10,000개) 세트 분할
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2857, random_state=42, stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 2. 개별 모델 훈련 (랜덤포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
ext_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = make_pipeline(StandardScaler(), SVC(kernel="linear", random_state=42))

# 모델 학습
rnd_clf.fit(X_train, y_train)
ext_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)

# 검증 세트에서 개별 모델 성능 확인
y_valid_pred_rf = rnd_clf.predict(X_valid)
y_valid_pred_ext = ext_clf.predict(X_valid)
y_valid_pred_svm = svm_clf.predict(X_valid)

print("RandomForest 정확도:", accuracy_score(y_valid, y_valid_pred_rf))
print("ExtraTrees 정확도:", accuracy_score(y_valid, y_valid_pred_ext))
print("SVM 정확도:", accuracy_score(y_valid, y_valid_pred_svm))


RandomForest 정확도: 0.9640964096409641
ExtraTrees 정확도: 0.9672967296729673
SVM 정확도: 0.9204920492049204


In [ ]:
# 3. 앙상블 분류기 생성 (Soft Voting) => 간접 투표 방식
voting_clf = VotingClassifier(
    estimators=[
        ("rf", rnd_clf),
        ("ext", ext_clf),
        ("svm", svm_clf)
    ],
    voting="hard"  # soft voting: 각 모델의 확률값 평균을 사용
)

# 앙상블 모델 학습
voting_clf.fit(X_train, y_train)

# 검증 세트에서 성능 평가
y_valid_pred_voting = voting_clf.predict(X_valid)
print("앙상블 정확도:", accuracy_score(y_valid, y_valid_pred_voting))


앙상블 정확도: 0.9664966496649665
